In [1]:
#Modificando Configuración
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1637936395482_0011,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
unknown magic command '%configure'
UnknownMagic: unknown magic command '%configure'



In [2]:
#import SparkSession
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.functions import desc
import functools

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#create spark session object
spark=SparkSession.builder.appName('data_processing_lab4_adchavesp').getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Load CSV COVID dataset
df=spark.sql("SELECT * FROM hue__tmp_covid19")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Los 10 departamentos con más casos de covid en Colombia ordenados de mayor a menor.

In [5]:
dfDep=spark.sql("SELECT `nombre departamento`, COUNT(*)  AS NumeroCasos FROM `default`.`hue__tmp_covid19` GROUP BY `nombre departamento` ORDER BY NumeroCasos DESC")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
dfDep.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+
|nombre departamento|NumeroCasos|
+-------------------+-----------+
|             BOGOTA|    1460569|
|          ANTIOQUIA|     765648|
|              VALLE|     415000|
|       CUNDINAMARCA|     265897|
|          SANTANDER|     229051|
|       BARRANQUILLA|     214692|
|          CARTAGENA|     125933|
|          ATLANTICO|     122465|
|             TOLIMA|     108010|
|             BOYACA|     106243|
+-------------------+-----------+
only showing top 10 rows

### Las 10 ciudades con más casos de covid en Colombia ordenados de mayor a menor.

In [7]:
dfMun=spark.sql("SELECT `nombre municipio`, COUNT(*)  AS NumeroCasos FROM `default`.`hue__tmp_covid19` GROUP BY `nombre municipio` ORDER BY NumeroCasos DESC")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
dfMun.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------+
|nombre municipio|NumeroCasos|
+----------------+-----------+
|          BOGOTA|    1460569|
|        MEDELLIN|     420064|
|            CALI|     286145|
|    BARRANQUILLA|     214692|
|       CARTAGENA|     125933|
|     BUCARAMANGA|     108712|
|          IBAGUE|      75261|
|     SANTA MARTA|      70195|
|       MANIZALES|      69820|
|           BELLO|      63372|
+----------------+-----------+
only showing top 10 rows

### Cambiando nombre de las columnas y convirtiendo un DataFrame en una vista temporal SQL.

In [9]:
#Cambiando nombre de las columnas
df2 = df.toDF('fecha reporte web', 'ID de caso', 'Fecha de notificacion', 'Codigo DIVIPOLA departamento', 'Nombre departamento', 'Codigo DIVIPOLA municipio', 'Nombre municipio', 'Edad', 'Unidad de medida de edad', 'Sexo', 'Tipo de contagio', 'Ubicacion del caso', 'Estado', 'Codigo ISO del pais', 'Nombre del pais', 'Recuperado', 'Fecha de inicio de sintomas', 'Fecha de muerte', 'Fecha de diagnostico', 'Fecha de recuperacion', 'Tipo de recuperacion', 'Pertenencia etnica', 'Nombre del grupo etnico')
df2.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['fecha reporte web', 'ID de caso', 'Fecha de notificacion', 'Codigo DIVIPOLA departamento', 'Nombre departamento', 'Codigo DIVIPOLA municipio', 'Nombre municipio', 'Edad', 'Unidad de medida de edad', 'Sexo', 'Tipo de contagio', 'Ubicacion del caso', 'Estado', 'Codigo ISO del pais', 'Nombre del pais', 'Recuperado', 'Fecha de inicio de sintomas', 'Fecha de muerte', 'Fecha de diagnostico', 'Fecha de recuperacion', 'Tipo de recuperacion', 'Pertenencia etnica', 'Nombre del grupo etnico']

In [10]:
# Register the DataFrame as a SQL temporary view
df2.createOrReplaceTempView("covid19")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Los 10 días con más casos de covid en Colombia ordenados de mayor a menor.

In [11]:
dfFechDiag=spark.sql("SELECT `Fecha de diagnostico`, COUNT(*) AS DiagnosticoPorDia FROM covid19 GROUP BY `Fecha de diagnostico` ORDER BY DiagnosticoPorDia DESC")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
dfFechDiag.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+
|Fecha de diagnostico|DiagnosticoPorDia|
+--------------------+-----------------+
|   25/6/2021 0:00:00|            34169|
|   23/6/2021 0:00:00|            33920|
|   15/6/2021 0:00:00|            33811|
|   24/6/2021 0:00:00|            33217|
|   21/6/2021 0:00:00|            33068|
|   16/6/2021 0:00:00|            32478|
|    9/6/2021 0:00:00|            32438|
|   29/6/2021 0:00:00|            32092|
|   11/6/2021 0:00:00|            31949|
|    8/6/2021 0:00:00|            31850|
+--------------------+-----------------+
only showing top 10 rows

### Distribución de casos por edades de covid en Colombia.

In [13]:
dfEdades = spark.sql("SELECT `Edad`, COUNT(*) AS CasosPorEdad FROM covid19 GROUP BY `Edad` ORDER BY CasosPorEdad DESC")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
dfEdades.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------+
|Edad|CasosPorEdad|
+----+------------+
|  30|      123529|
|  28|      122613|
|  29|      122536|
|  27|      120926|
|  26|      120688|
|  31|      120421|
|  32|      117559|
|  25|      115960|
|  33|      113163|
|  35|      112485|
|  34|      111369|
|  24|      110740|
|  36|      110503|
|  38|      108352|
|  37|      106948|
|  39|      106470|
|  40|      104821|
|  23|      102041|
|  41|      100419|
|  22|       93349|
+----+------------+
only showing top 20 rows

 ### Pregunta abierta: ¿10 Dias en los que falleció más gente?

In [17]:
dfMuertes = spark.sql("SELECT `Fecha de muerte`, COUNT(*) AS MuertesPorDia FROM covid19 GROUP BY `Fecha de muerte` ORDER BY MuertesPorDia DESC NULLS LAST")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
dfMuertes.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------------+
|  Fecha de muerte|MuertesPorDia|
+-----------------+-------------+
|                 |      4901329|
|21/6/2021 0:00:00|          716|
|24/6/2021 0:00:00|          694|
|20/6/2021 0:00:00|          684|
|29/6/2021 0:00:00|          682|
|23/6/2021 0:00:00|          682|
|14/6/2021 0:00:00|          679|
|19/6/2021 0:00:00|          674|
|17/6/2021 0:00:00|          674|
|13/6/2021 0:00:00|          673|
+-----------------+-------------+
only showing top 10 rows